In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import os

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

%matplotlib inline
%load_ext google.cloud.bigquery

In [ ]:
from google.cloud import bigquery

credential_path = r'../../BQ_Keypath/portfolio-331917-9f920a11ce56.json'
client = bigquery.Client.from_service_account_json(credential_path)

# define helper functions
def getQueryAsDF(query):
    return client.query(query).result().to_dataframe()

def getTableSchema(table):
    return client.get_table(table).schema

## 1. Population Growth Over Time
We will be looking at users signup rate year by year. This will give us insight for the platform growth. 

In [ ]:
# understanding users table's schema
schema = getTableSchema('bigquery-public-data.stackoverflow.users')
schema

In [ ]:
# since all field are NULLABLE modes, checking for nulls 
query = """
            SELECT 
                COUNT(*) AS null_count
            FROM
                `bigquery-public-data.stackoverflow.users`
            WHERE
                id IS NULL
                OR creation_date IS NULL
                OR last_access_date IS NULL;
        """

result = getQueryAsDF(query)
result

In [ ]:
# user signup count by year
query = """
            SELECT 
                EXTRACT (YEAR FROM creation_date) AS year,
                COUNT(*) AS signup_count
            FROM `bigquery-public-data.stackoverflow.users`
            GROUP BY 1
        """

result = getQueryAsDF(query)
result

In [ ]:
# looking at user signup trend yearly.
users_count = result['signup_count'].values
total_user_count = result['signup_count'].sum()
cumulative = []
cumulative.append(round(users_count[0]/total_user_count * 100, 2))

for i in range(len(users_count)):
    index = i+2
    if index <= len(users_count):
        curr_count = np.sum(users_count[:index])
        cumulative.append(round(curr_count/total_user_count * 100, 2))
        
df = result
df['cumulative'] = cumulative
df

In [ ]:
fig, ax = plt.subplots();

ax = sns.barplot(data = df, x = 'year', y = 'signup_count', palette = 'Blues')

avg_growth_count = np.average(df['signup_count'])
text_avg_growth_count = 'Average signup: {}'.format(round(avg_growth_count))
plt.axhline(avg_growth_count, ls = '--')
plt.text(0, avg_growth_count * 1.1, text_avg_growth_count)
plt.xticks(rotation = 45)
plt.ylabel('signup count by x1 million')

ax2 = ax.twinx()
ax2.plot(df.index, df['cumulative'], color = 'orange', marker="D", ms=5)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylabel("Cumulative Percentage");

plt.title('New User Signup Count')

Population growth for the platform is growing year after year with an average of 1109925. We can see the spurred starting from 2013. Each year there after gained by 1x million. 

## 2. MAU KPI. 
- Monthly Active Users(MAU): unique user post by comment, question, answer or accessed site(signed in).

In [ ]:
# looking into comments, question and answer tables schema
schema = getTableSchema('bigquery-public-data.stackoverflow.posts_questions')
schema

In [ ]:
schema = getTableSchema('bigquery-public-data.stackoverflow.posts_answers')
schema

In [ ]:
schema = getTableSchema('bigquery-public-data.stackoverflow.comments')
schema

Can safely assume the creation_date field for each of the table has non-null values.

In [ ]:
from google.cloud.exceptions import NotFound

# creating a view for the consolidated MAU for quick accesses later on.
view_id = "portfolio-331917.stored_views.active_users"

try:
    client.get_table(view_id)
    print('view already exist, skipping')
except NotFound:
    view = bigquery.Table(view_id)
    view.view_query = f"""
                        SELECT
                            EXTRACT (YEAR FROM creation_date) AS year,
                            EXTRACT (MONTH FROM creation_date) AS month,
                            owner_user_id
                        FROM `bigquery-public-data.stackoverflow.posts_questions`
                        UNION ALL 
                        SELECT
                            EXTRACT (YEAR FROM creation_date) AS year,
                            EXTRACT (MONTH FROM creation_date) AS month,
                            owner_user_id
                        FROM `bigquery-public-data.stackoverflow.posts_answers`
                        UNION ALL 
                        SELECT
                            EXTRACT (YEAR FROM creation_date) AS year,
                            EXTRACT (MONTH FROM creation_date) AS month,
                            user_id AS owner_user_id
                        FROM `bigquery-public-data.stackoverflow.comments`
                        UNION ALL
                        SELECT
                            EXTRACT (YEAR FROM last_access_date) AS year,
                            EXTRACT (MONTH FROM last_access_date) AS month,
                            id AS owner_user_id
                        FROM `bigquery-public-data.stackoverflow.users`
                        """
    view = client.create_table(view)

In [ ]:
# exclusion of 2021 due to incomplete data.
query = """
        Select
            year,
            month,
            COUNT(DISTINCT owner_user_id) AS active_users
        FROM `portfolio-331917.stored_views.active_users`
        Where
            year < 2021 -- no difference applying filter before or after agg
        GROUP BY 1,2
        ORDER BY 1,2;
        """

result = getQueryAsDF(query)
result.head()

In [ ]:
result.describe()
# average active users making comment, answer and question post overall is 205618.620000 a month.

In [ ]:
query = """
        Select
            month,
            ROUND(COUNT(DISTINCT owner_user_id) / COUNT(DISTINCT CONCAT(year, month))) AS avg_mau
        FROM `portfolio-331917.stored_views.active_users`
        WHERE
            year < 2021
        GROUP BY 1
        ORDER BY 1;
        """

result = getQueryAsDF(query)
result

In [ ]:
sns.lineplot(data = result, x = 'month', y = 'avg_mau')
plt.ylabel('average MAU')
plt.title('Overall Average of MAU < 2021')

At a glance, this is a bimodal distribution with a right skew. 

Interestingly, from the 2 plots, average MAU peaks out in May, dips during the summer months and rebounds in October to November. College summer and winter breaks for might be a contributing factor. 

In [ ]:
query = """
        WITH t1 AS
        (
            Select
                month,
                ROUND(COUNT(DISTINCT owner_user_id) / COUNT(DISTINCT CONCAT(year, month))) AS avg_mau
            FROM `portfolio-331917.stored_views.active_users`
            WHERE
                year < 2021
            GROUP BY 1
            ORDER BY 1
        ),
        
        t2 AS
        (
            Select
                month,
                COUNT(DISTINCT owner_user_id) as active_user_count,
            FROM `portfolio-331917.stored_views.active_users`
            WHERE
                year = 2021
            GROUP BY 1
            ORDER BY 1
        )
        
        SELECT
            t1.month,
            active_user_count,
            avg_mau
        FROM t1
        INNER JOIN t2 ON t1.month = t2.month
        ORDER BY 1;
        """

result = getQueryAsDF(query)
result

Clearly, the raw MAU for 2021 is ~4-10x higher than the overal average. 2021 MAU might be an outlier in the overal population distribution. 